# **Feature Engineering Notebook**

## Objectives

*   Evaluate missing data
*   Clean data

## Inputs

* outputs/datasets/collection/HousePricePredictionSales.csv

## Outputs

* Generate cleaned Train and Test sets, both saved under outputs/datasets/cleaned
  
## Conclusions

  * Data Cleaning Pipeline
  * Drop Variables: ['EnclosedPorch', 'WoodDeckSF' ]




---

# Change working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [2]:
import os
current_dir = os.getcwd()
current_dir

'/workspaces/milestone-project-heritage-housing/jupyter_notebooks'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [3]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [4]:
current_dir = os.getcwd()
current_dir

'/workspaces/milestone-project-heritage-housing'

---

# Load Collected data

In [5]:
import pandas as pd
df = (pd.read_csv("outputs/datasets/collection/HousePricePredictionSales.csv")
     )
df.head(3)

1stFlrSF  2ndFlrSF  BedroomAbvGr BsmtExposure  BsmtFinSF1 BsmtFinType1  \
0       856     854.0           3.0           No         706          GLQ   
1      1262       0.0           3.0           Gd         978          ALQ   
2       920     866.0           3.0           Mn         486          GLQ   

   BsmtUnfSF  EnclosedPorch  GarageArea GarageFinish  ...  LotFrontage  \
0        150            0.0         548          RFn  ...         65.0   
1        284            NaN         460          RFn  ...         80.0   
2        434            0.0         608          RFn  ...         68.0   

   MasVnrArea OpenPorchSF  OverallCond  OverallQual  TotalBsmtSF  WoodDeckSF  \
0       196.0          61            5            7          856         0.0   
1         0.0           0            8            6         1262         NaN   
2       162.0          42            5            7          920         NaN   

   YearBuilt  YearRemodAdd  SalePrice  
0       2003          2003     208500  
1       1976          1976     181500  
2       2001          2002     223500  

[3 rows x 24 columns]

---

# Data Exploration

We are interested to check the distribution and shape of a variable with missing data.

In [6]:
vars_with_missing_data = df.columns[df.isna().sum() > 0].to_list()
vars_with_missing_data

['2ndFlrSF',
 'BedroomAbvGr',
 'BsmtFinType1',
 'EnclosedPorch',
 'GarageFinish',
 'GarageYrBlt',
 'LotFrontage',
 'MasVnrArea',
 'WoodDeckSF']

In [7]:
from ydata_profiling import ProfileReport
if vars_with_missing_data:
    profile = ProfileReport(df=df[vars_with_missing_data], minimal=True)
    profile.to_notebook_iframe()
else:
    print("There are no variables with missing data")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

---

# Data Cleaning

## Assessing Missing Data Levels

In [8]:
def EvaluateMissingData(df):
    missing_data_absolute = df.isnull().sum()
    missing_data_percentage = round(missing_data_absolute/len(df)*100, 2)
    df_missing_data = (pd.DataFrame(
                            data={"RowsWithMissingData": missing_data_absolute,
                                   "PercentageOfDataset": missing_data_percentage,
                                   "DataType": df.dtypes}
                                    )
                          .sort_values(by=['PercentageOfDataset'], ascending=False)
                          .query("PercentageOfDataset > 0")
                          )

    return df_missing_data


Check missing data levels for the collected dataset.

In [9]:
EvaluateMissingData(df)

RowsWithMissingData  PercentageOfDataset DataType
EnclosedPorch                 1324                90.68  float64
WoodDeckSF                    1305                89.38  float64
LotFrontage                    259                17.74  float64
GarageFinish                   162                11.10   object
BsmtFinType1                   114                 7.81   object
BedroomAbvGr                    99                 6.78  float64
2ndFlrSF                        86                 5.89  float64
GarageYrBlt                     81                 5.55  float64
MasVnrArea                       8                 0.55  float64

## Dealing with Missing Data

### Data Cleaning Summary

List here the data cleaning approaches you want initially to try.
* Fill the numerical missing data with 'mean'
* Encode de "object" variables 
* Drop - `['WoodDeckSF', 'EnclosedPorch']`

### Encode object variables:

In [14]:
var_enc = {'BsmtExposure': {'None': 0, 'No': 1, 'Mn': 2, 'Av': 3, 'Gd': 4}, 'BsmtFinType1': {'None': 0, 'Unf': 1, 'LwQ': 2, 'Rec': 3, 'BLQ': 4, 'ALQ': 5, 'GLQ': 6}, 
           'GarageFinish': {'None': 0, 'Unf': 1, 'RFn': 2, 'Fin': 3}, 'KitchenQual': {'Po': 0, 'Fa': 1, 'TA': 2, 'Gd': 3, 'Ex': 4}}
df_en=df.copy()
for col in df.columns[df.dtypes=='object'].to_list():
    df_en[col] = df_en[col].replace(var_enc[col])
df_en.head()

1stFlrSF  2ndFlrSF  BedroomAbvGr  BsmtExposure  BsmtFinSF1  BsmtFinType1  \
0       856     854.0           3.0             1         706           6.0   
1      1262       0.0           3.0             4         978           5.0   
2       920     866.0           3.0             2         486           6.0   
3       961       NaN           NaN             1         216           5.0   
4      1145       NaN           4.0             3         655           6.0   

   BsmtUnfSF  EnclosedPorch  GarageArea  GarageFinish  ...  LotFrontage  \
0        150            0.0         548           2.0  ...         65.0   
1        284            NaN         460           2.0  ...         80.0   
2        434            0.0         608           2.0  ...         68.0   
3        540            NaN         642           1.0  ...         60.0   
4        490            0.0         836           2.0  ...         84.0   

   MasVnrArea  OpenPorchSF  OverallCond  OverallQual  TotalBsmtSF  WoodDeckSF  \
0       196.0           61            5            7          856         0.0   
1         0.0            0            8            6         1262         NaN   
2       162.0           42            5            7          920         NaN   
3         0.0           35            5            7          756         NaN   
4       350.0           84            5            8         1145         NaN   

   YearBuilt  YearRemodAdd  SalePrice  
0       2003          2003     208500  
1       1976          1976     181500  
2       2001          2002     223500  
3       1915          1970     140000  
4       2000          2000     250000  

[5 rows x 24 columns]

### Split Train and Test Set

In [15]:
from sklearn.model_selection import train_test_split
TrainSet, TestSet, _, __ = train_test_split(
                                        df,
                                        df['SalePrice'],
                                        test_size=0.2,
                                        random_state=0)

print(f"TrainSet shape: {TrainSet.shape} \nTestSet shape: {TestSet.shape}")

TrainSet shape: (1168, 24) 
TestSet shape: (292, 24)


In [16]:
df_missing_data = EvaluateMissingData(TrainSet)
print(f"* There are {df_missing_data.shape[0]} variables with missing data \n")
df_missing_data

* There are 9 variables with missing data 



RowsWithMissingData  PercentageOfDataset DataType
EnclosedPorch                 1056                90.41  float64
WoodDeckSF                    1034                88.53  float64
LotFrontage                    212                18.15  float64
GarageFinish                   131                11.22   object
BsmtFinType1                    89                 7.62   object
BedroomAbvGr                    80                 6.85  float64
2ndFlrSF                        60                 5.14  float64
GarageYrBlt                     58                 4.97  float64
MasVnrArea                       6                 0.51  float64

In [17]:
from sklearn.base import BaseEstimator, TransformerMixin
# create a Class variable with a fit and transform method
class MyCustomEncoder(BaseEstimator, TransformerMixin):

  def __init__(self, variables, dic):
    if not isinstance(variables, list): 
      self.variables = [variables]
    else: self.variables = variables
    self.var_enc = var_enc

  def fit(self, X, y=None):    
    return self

  def transform(self, X):
    for col in self.variables:
      if X[col].dtype == 'object':
        X[col] = X[col].replace(var_enc[col])
      else:
        print(f"Warning: {col} data type should be object to use MyCustomEncoder()")
      
    return X

# use the custom encoder in a pipeline
from sklearn.pipeline import Pipeline
pipeline = Pipeline([('custom_encoder', MyCustomEncoder(variables=['BsmtExposure', 'BsmtFinType1', 'GarageFinish', 
                                                                   'KitchenQual'], dic=var_enc))])



df_en = df.copy()
df_en = pipeline.fit_transform(df_en)
df_en.head(3)

1stFlrSF  2ndFlrSF  BedroomAbvGr  BsmtExposure  BsmtFinSF1  BsmtFinType1  \
0       856     854.0           3.0             1         706           6.0   
1      1262       0.0           3.0             4         978           5.0   
2       920     866.0           3.0             2         486           6.0   

   BsmtUnfSF  EnclosedPorch  GarageArea  GarageFinish  ...  LotFrontage  \
0        150            0.0         548           2.0  ...         65.0   
1        284            NaN         460           2.0  ...         80.0   
2        434            0.0         608           2.0  ...         68.0   

   MasVnrArea  OpenPorchSF  OverallCond  OverallQual  TotalBsmtSF  WoodDeckSF  \
0       196.0           61            5            7          856         0.0   
1         0.0            0            8            6         1262         NaN   
2       162.0           42            5            7          920         NaN   

   YearBuilt  YearRemodAdd  SalePrice  
0       2003          2003     208500  
1       1976          1976     181500  
2       2001          2002     223500  

[3 rows x 24 columns]

### Drop Variables 

Hint: you may drop Variables with more than 80% of missing data since these variables will likely not add much value. However, this is not the case in this dataset
Step 1: imputation approach: Drop Variables
Step 2: Select variables to apply the imputation approach

In [18]:
drop_method = ['WoodDeckSF', 'EnclosedPorch']

print(f"* {len(drop_method)} variables to drop \n\n"
    f"{drop_method}")

* 2 variables to drop 

['WoodDeckSF', 'EnclosedPorch']


* Step 3: Create a separate DataFrame applying this imputation approach to the selected variables.

In [20]:
# Remove 'EnclosedPorch' and 'WoodDeckSF' from vars_with_missing_dat since we drop them
vars_with_missing_data = ['2ndFlrSF', 'BedroomAbvGr', 'BsmtFinType1', 'GarageFinish', 'GarageYrBlt', 'LotFrontage', 'MasVnrArea']
from feature_engine.imputation import MeanMedianImputer
from feature_engine.selection import DropFeatures

pipeline = Pipeline([
      ('drop_features', DropFeatures(features_to_drop = drop_method)),
      
      ('median_imputer',  MeanMedianImputer(imputation_method='median', variables=vars_with_missing_data))
])

df2 = df.copy()
df_transformed = pipeline.fit_transform(df_en) 
df_transformed.head(5) 

1stFlrSF  2ndFlrSF  BedroomAbvGr  BsmtExposure  BsmtFinSF1  BsmtFinType1  \
0       856     854.0           3.0             1         706           6.0   
1      1262       0.0           3.0             4         978           5.0   
2       920     866.0           3.0             2         486           6.0   
3       961       0.0           3.0             1         216           5.0   
4      1145       0.0           4.0             3         655           6.0   

   BsmtUnfSF  GarageArea  GarageFinish  GarageYrBlt  ...  LotArea  \
0        150         548           2.0       2003.0  ...     8450   
1        284         460           2.0       1976.0  ...     9600   
2        434         608           2.0       2001.0  ...    11250   
3        540         642           1.0       1998.0  ...     9550   
4        490         836           2.0       2000.0  ...    14260   

   LotFrontage  MasVnrArea  OpenPorchSF  OverallCond  OverallQual  \
0         65.0       196.0           61            5            7   
1         80.0         0.0            0            8            6   
2         68.0       162.0           42            5            7   
3         60.0         0.0           35            5            7   
4         84.0       350.0           84            5            8   

   TotalBsmtSF  YearBuilt  YearRemodAdd  SalePrice  
0          856       2003          2003     208500  
1         1262       1976          1976     181500  
2          920       2001          2002     223500  
3          756       1915          1970     140000  
4         1145       2000          2000     250000  

[5 rows x 22 columns]

* Apply the transformation to the data.

In [21]:
vars_with_missing_data = ['2ndFlrSF', 'BedroomAbvGr', 'BsmtFinType1', 'GarageFinish', 'GarageYrBlt', 'LotFrontage', 'MasVnrArea']
from feature_engine.imputation import MeanMedianImputer
from feature_engine.selection import DropFeatures

pipeline = Pipeline([
      ('drop_features', DropFeatures(features_to_drop = drop_method)),
      ('custom_encoder', MyCustomEncoder(variables=['BsmtExposure', 'BsmtFinType1', 
                                                    'GarageFinish', 'KitchenQual'], dic=var_enc)),
      ('median_imputer',  MeanMedianImputer(imputation_method='median', variables=vars_with_missing_data))
])


df2 = df.copy()
df_transformed = pipeline.fit_transform(TrainSet) 




TrainSet, TestSet = pipeline.fit_transform(TrainSet) , pipeline.fit_transform(TestSet)

* Evaluate if you have more variables to deal with.

In [22]:
EvaluateMissingData(TrainSet)

Empty DataFrame
Columns: [RowsWithMissingData, PercentageOfDataset, DataType]
Index: []

---

# Push cleaned data to Repo

In [25]:
import os
try:
  os.makedirs(name='outputs/datasets/cleaned') # create outputs/datasets/collection folder
except Exception as e:
  print(e)

## Train Set

In [26]:
TrainSet.to_csv("outputs/datasets/cleaned/TrainSetCleaned.csv", index=False)

## Test Set

In [27]:
TestSet.to_csv("outputs/datasets/cleaned/TestSetCleaned.csv", index=False)